In [1]:
import numpy as np
import pandas as pd
import wandb
import pprint
import helper
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

In [2]:
# Preparing dataset
wandb.init(name='load_tweet_dataset_1',
                 project='epfl_ml_project2', 
                 tags=['load_dataset'],
                 job_type='for_testing',
                 anonymous='allow')
artifact = wandb.use_artifact('hsunyu/epfl_ml_project2/twitter_dataset_1:v0')
artifact_dir = artifact.download()
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: cr7_reunited (cr7_reunited-cr7). Use `wandb login --relogin` to force relogin


wandb:   3 of 3 files downloaded.  


In [3]:
wandb.login()

True

In [4]:
# Load it into a pandas dataframe
train_df = pd.read_json(artifact_dir + '/train.json', lines=True)
val_df = pd.read_json(artifact_dir + '/val.json', lines=True)
test_df = pd.read_json(artifact_dir + '/test.json', lines=True)

In [5]:
# Initializing hyperparameter grid
sweep_config = {
    'method': 'grid'
}

metric = {
    'name': 'val_accuracy',
    'goal': 'maximize'
}

parameters_dict = {
    'dataset': {
        'values': ['bow', 'tfidf']
    },
    'num_features': {
        'values': [100, 500, 1000, 5000, 10000, 20000, 50000, 74552]
    },
    'regularizer_strength': {
        'values': [0.01, 0.1, 1, 10, 100]
    }
}

sweep_config['metric'] = metric
sweep_config['parameters'] = parameters_dict

In [6]:
pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'maximize', 'name': 'val_accuracy'},
 'parameters': {'dataset': {'values': ['bow', 'tfidf']},
                'num_features': {'values': [100,
                                            500,
                                            1000,
                                            5000,
                                            10000,
                                            20000,
                                            50000,
                                            74552]},
                'regularizer_strength': {'values': [0.01, 0.1, 1, 10, 100]}}}


In [7]:
sweep_id = wandb.sweep(sweep_config, project='epfl_ml_project2')

Create sweep with ID: ebsgyfi0
Sweep URL: https://wandb.ai/cr7_reunited-cr7/epfl_ml_project2/sweeps/ebsgyfi0


In [8]:
# Preprocessing dataset, `datasets` is a dictionary of keys 'bow', 'tfidf' with values containing their respective train, validation, and test datasets
datasets = helper.prepare_datasets(train_df, val_df, test_df)

In [9]:
# Getting optimal Random Forest's hyperparameters
best_RF_params = helper.get_optimal_RF_params()

wandb: Sorting runs by -summary_metrics.val_accuracy


In [10]:
# Training optimal random forest
X_train, _, X_test, y_train, _, y_test = datasets[best_RF_params['dataset']]
clf = RandomForestClassifier(n_estimators=best_RF_params['n_estimators'], criterion=best_RF_params['criterion'], max_depth=best_RF_params['max_depth'], min_samples_leaf=best_RF_params['min_samples_leaf'], n_jobs=-1, random_state=599131614)
clf.fit(X_train, y_train)

RandomForestClassifier(criterion='log_loss', n_estimators=500, n_jobs=-1,
                       random_state=599131614)

In [11]:
# Generating feature importances from optimal random forest
importances = clf.feature_importances_
print(importances.shape)
indices = np.argsort(importances)[::-1]

(74552,)


In [12]:
def train(config=None):
    """
    Boilerplate for training and evaluating model from hyperparameter grids using wandb.
    Boilerplate for each model differes a little bit, hence needs to be manually configured for each model.
    """
    with wandb.init(config=config):
        config = wandb.config
        X_train, X_val,_ , y_train, y_val, _ = helper.get_features(*datasets[config.dataset], config.num_features, indices)
        clf = LogisticRegression(C=config.regularizer_strength, random_state=599131614, max_iter=1000)
        clf.fit(X_train, y_train)
        wandb.log({'train_accuracy': accuracy_score(y_train, clf.predict(X_train)), 'val_accuracy': accuracy_score(y_val, clf.predict(X_val))})

In [13]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 3kxo9gif with config:
wandb: 	dataset: bow
wandb: 	num_features: 100
wandb: 	regularizer_strength: 0.01


train_accuracy,▁
val_accuracy,▁
train_accuracy,0.72286
val_accuracy,0.71293


wandb: Agent Starting Run: fqxqmpz8 with config:
wandb: 	dataset: bow
wandb: 	num_features: 100
wandb: 	regularizer_strength: 0.1


train_accuracy,▁
val_accuracy,▁
train_accuracy,0.72508
val_accuracy,0.71415


wandb: Agent Starting Run: 0mp2aqkt with config:
wandb: 	dataset: bow
wandb: 	num_features: 100
wandb: 	regularizer_strength: 1


...

train_accuracy,▁
val_accuracy,▁
train_accuracy,0.89959
val_accuracy,0.79945


wandb: Agent Starting Run: 38aocwj2 with config:
wandb: 	dataset: tfidf
wandb: 	num_features: 74552
wandb: 	regularizer_strength: 100


/home/parkmyungheon/miniconda3/envs/ADA/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


train_accuracy,▁
val_accuracy,▁
train_accuracy,0.92053
val_accuracy,0.78143


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [16]:
def train_test(config):
    """
    Boilerplate for training and testing optimal model.
    Boilerplate for each model differes a little bit, hence need to be manually configured for each model.
    Params:
    config: a dictionary containing optimal hyperparameters
    """
    X_train, X_val, X_test, y_train, y_val, y_test = helper.get_features(
        *datasets[config['dataset']],
        config['num_features'],
        indices
    )
    clf = LogisticRegression(C=config['regularizer_strength'], random_state=599131614, max_iter=1000)
    clf.fit(X_train, y_train)
    print(f'train accuracy: {accuracy_score(y_train, clf.predict(X_train))}')
    print(f'val accuracy: {accuracy_score(y_val, clf.predict(X_val))}')
    print(f'test accuracy: {accuracy_score(y_test, clf.predict(X_test))}')
    print(f'test f1-score: {f1_score(y_test, clf.predict(X_test))}')

In [17]:
# Training and testing optimal model for TF-IDF dataset
conditions = {
    'dataset': 'tfidf'
}

best_params = helper.get_best_run_by_conditions(sweep_id, conditions)
print(best_params)
train_test(best_params)

{'regularizer_strength': 1, 'dataset': 'tfidf', 'num_features': 74552}
train accuracy: 0.847275
val accuracy: 0.80315
test accuracy: 0.806825
test f1-score: 0.8086665841277703


In [18]:
# Training and testing optimal model for BoW dataset
conditions = {
    'dataset': 'bow'
}

best_params = helper.get_best_run_by_conditions(sweep_id, conditions)
print(best_params)
train_test(best_params)

{'dataset': 'bow', 'num_features': 74552, 'regularizer_strength': 1}
train accuracy: 0.8767166666666667
val accuracy: 0.80155
test accuracy: 0.80655
test f1-score: 0.8116083166966939
